<a href="https://colab.research.google.com/github/nogbazghi/CSC8980/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nahom Ogbazghi

Create a cosine similarity matrix for all of Shakespeare’s works found in the provided
file. This will result in a 42 by 42 matrix with the cosine similarity between each of his
works. In other words, calculate the document-wise cosine similarity between all of
Shakepeare’s works. (50 points). Use TF_IDF for this. Note, you can use the Cosine
Similarity function on scikit-learn or implement your own, but no other library/package is
allowed

In [ ]:
import os
import re
import math
import sklearn.metrics
dir = "/content"
matrix = {}
N=42
allFilenames = []
for filename in os.listdir(os.getcwd()):
  if filename.endswith(".txt"):
    allFilenames.append(filename)
    file = open("/content/"+str(filename), 'r')
    content = file.read()
    getWords = re.compile("[a-zA-Z]+")
    words = re.findall(getWords, content)
    for anyCaseWord in words:
      word = anyCaseWord.lower()
      if word in matrix:
        wcForFilename = matrix[word]
        if filename in wcForFilename:
          count = wcForFilename[filename]
          wcForFilename[filename] = count + 1
          matrix[word] = wcForFilename
        else:
          wcForFilename[filename] = 1
          matrix[word] = wcForFilename
      else:
        newFileWordPair = {}
        newFileWordPair[filename] = 1
        matrix[word] = newFileWordPair

def find_tf_id(count):
  return math.log(count+1, 10)

def find_df(matrix, word):
  docs = matrix[word]
  return len(docs.keys())

def find_idf(matrix, word):
  df = find_df(matrix, word)
  count = 42/df
  return math.log(count, 10)

def find_tf_idf(tf, idf):
  return tf * idf

# idf per word
idf_matrix = {}
words = matrix.keys()
for word in words:
  idf = find_idf(matrix, word)
  idf_matrix[word] = idf

print("all words", len(words))
# tf idf per word
tf_idf_matrix = {} #key:filename, value: {key:word, value:tf idf} 
for word in words:
  wcPerFileForWord = matrix[word]
  filenamesOfWord = wcPerFileForWord.keys()
  for filename in filenamesOfWord:
    count = wcPerFileForWord[filename]
    tf = find_tf_id(count)
    idf = idf_matrix[word]
    if filename in tf_idf_matrix:
      tfsPerWord = tf_idf_matrix[filename]
    else:
      tfsPerWord = {}
    tfsPerWord[word] = find_tf_idf(tf, idf)
  tf_idf_matrix[filename] = tfsPerWord

# put 0s for words not seen
for f in allFilenames:
  usedWords = tf_idf_matrix[f].keys()
  word_tfidf_pairs = tf_idf_matrix[f]
  for word in words:
    if word not in usedWords:
      word_tfidf_pairs[word] = 0
  tf_idf_matrix[f] = word_tfidf_pairs

def cosSimOfTwoDocs(fileA, fileB):
  fileAPairs = tf_idf_matrix[fileA]
  fileBPairs = tf_idf_matrix[fileB]
  num = 0
  den = 0
  aDen = 0
  bDen = 0
  for word in words:
    numMult = fileAPairs[word] * fileBPairs[word] 
    denASquare = math.pow(fileAPairs[word], 2)
    denBSquare = math.pow(fileBPairs[word], 2)
    num += numMult #A*B
    aDen += denASquare #|A|
    bDen += denBSquare #|B|
  den = math.sqrt(aDen) * math.sqrt(bDen) #|A||B|
  return num/den #A*B/|A||B|

cosDocPairs = {}
for fileA in allFilenames:
  cosSimVals = {}
  filesinCosDocPairs = cosDocPairs.keys()
  if fileA in filesinCosDocPairs:
    cosSimVals = cosDocPairs[fileA]
  for fileB in allFilenames:
    cosSim = cosSimOfTwoDocs(fileA, fileB)
    cosSimVals[fileB] = cosSim
  cosDocPairs[fileA] = cosSimVals

print(cosDocPairs)

all words 24483
{'the-two-noble-kinsmen_TXT_FolgerShakespeare.txt': {'the-two-noble-kinsmen_TXT_FolgerShakespeare.txt': 0.9999999999999998, 'coriolanus_TXT_FolgerShakespeare.txt': 0.05985951859060359, 'henry-iv-part-1_TXT_FolgerShakespeare.txt': 0.07689263792786862, 'king-john_TXT_FolgerShakespeare.txt': 0.07207474240296737, 'king-lear_TXT_FolgerShakespeare.txt': 0.09060271702459764, 'henry-v_TXT_FolgerShakespeare.txt': 0.05483268094270949, 'henry-vi-part-1_TXT_FolgerShakespeare.txt': 0.07394925660209752, 'a-midsummer-nights-dream_TXT_FolgerShakespeare.txt': 0.11165886793312828, 'pericles_TXT_FolgerShakespeare.txt': 0.06329402854423337, 'the-comedy-of-errors_TXT_FolgerShakespeare.txt': 0.0, 'othello_TXT_FolgerShakespeare.txt': 0.07659411781518725, 'the-merry-wives-of-windsor_TXT_FolgerShakespeare.txt': 0.021367160893519293, 'venus-and-adonis_TXT_FolgerShakespeare.txt': 0.07292914173566133, 'the-phoenix-and-turtle_TXT_FolgerShakespeare.txt': 0.013249262219830433, 'troilus-and-cressida_T

2. Write a function that takes the previous matrix and a number n as parameters (nothing else will be accepted) and return the top n similar works. Use the function to output the top 10 similar works. (30 points).

In [ ]:
def replacedSmallest(m, num, nameA, nameB):
  values = list(m.values())
  values.sort()
  smallest = values[0]
  if(num > smallest):
    names = m.keys()
    for n in names:
      if m[n] == smallest:
        m.pop(n)
        m[(nameA,nameB)] = num
  return m

def alreadyInList(m, nameA, nameB):
  names = m.keys()
  for n in names:
    if nameA in n:
      if nameB in n:
        return True
  return False

def topNSimilarWorks(n):
  nWorks = {}
  for fA in allFilenames:
    cosList = cosDocPairs[fA]
    for fB in allFilenames:
      if alreadyInList(nWorks, fA, fB) == False: #already seen
        if fA != fB: # same pair
          value = cosList[fB]
          if len(nWorks) == 10:
            nWorks = replacedSmallest(nWorks, value, fA, fB)
          else:
            nWorks[(fA,fB)] = value
  return nWorks.keys()

items = topNSimilarWorks(10)
print("Top", len(items), "similar files")
for item in items:
  first = cosDocPairs[item[0]]
  print ((item[0], item[1]))

Top 10 similar files
('comp.os.ms-windows.misc', 'comp.sys.mac.hardware')
('comp.os.ms-windows.misc', 'rec.autos')
('comp.os.ms-windows.misc', 'talk.politics.guns')
('comp.os.ms-windows.misc', 'talk.politics.mideast')
('comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware')
('comp.sys.ibm.pc.hardware', 'rec.autos')
('comp.sys.ibm.pc.hardware', 'sci.electronics')
('comp.sys.ibm.pc.hardware', 'talk.religion.misc')
('comp.sys.mac.hardware', 'sci.med')
('rec.motorcycles', 'rec.sport.baseball')


3. Using the code from the Language Models II class, train two simple language models using all of the files (together) in shakespeares-works_TXT_FolgerShakespeare.zip. One model should be trained using bigrams, the other using trigrams. (40 points).

In [138]:
import nltk
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

def trigramModel(allFilenames):
  model = defaultdict(lambda: defaultdict(lambda: 0))
  for filename in allFilenames:
    file = open("/content/"+str(filename), 'r')
    for line in file:
      sentence = line.split()
      for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1

  # Let's transform the counts to probabilities
  for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count
  return model

def bigramModel(allFilenames):
  model = defaultdict(lambda: defaultdict(lambda: 0))
  for filename in allFilenames:
    file = open("/content/"+str(filename), 'r')
    for line in file:
      sentence = line.split()
      for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        model[w1][w2] += 1

  # Let's transform the counts to probabilities
  for w1 in model:
    total_count = float(sum(model[w1].values()))
    for w2 in model[w1]:
        model[w1][w2] /= total_count
  return model

# Run trimodel and bimodels seperately, results are not good when run together
print("trigram")
trimodel = trigramModel(allFilenames)
print("I will")
dict(trimodel['I', 'will'])

# print("bigram")
# bimodel = bigramModel(allFilenames)
# print("I")
# dict(bimodel['I'])


trigram


FileNotFoundError: ignored

4. Write a function that takes the following three parameters: model, list of start words, number of sentences to generate. This function should return the sentences generated as a list. DO NOT print anything to the screen from within the function. Use this function to generate 10 sentences with the bigram model from the previous question, and 5 sentences with the trigram model from the previous question. (30 points)

In [ ]:
import random
from random import randrange

def generateSentences(model, startWords, count):
  sentences = []
  modelType = list(model.keys())[0]
  sentLength = 0 
  if type(modelType) is not tuple:
    while sentLength < count:
      startIndex = randrange(len(startWords))
      text = [startWords[startIndex]]
      sentence_finished = False
      while not sentence_finished:
        # select a random probability threshold  
        r = random.random()
        accumulator = .0
        n = -1
        for word in model[words[n:][0]].keys():
          accumulator += model[words[n:][0]][word]
          # select words that are above the probability threshold
          if accumulator >= r:
              text.append(word)
              break
        if text[n:] == [None]:
            sentence_finished = True
            sentence = (' '.join([t for t in text if t]))
            sentences.append(sentence)
            sentLength+=1
    return sentences
  elif type(modelType) is tuple: #trigram
    while sentLength < count:
      startIndex = randrange(len(startWords)-1)
      text = [startWords[startIndex], startWords[startIndex+1]]
      sentence_finished = False
      while not sentence_finished:
        # select a random probability threshold  
        r = random.random()
        accumulator = .0
        for word in model[tuple(text[-2:])].keys():
          accumulator += model[tuple(text[-2:])][word]
          # select words that are above the probability threshold
          if accumulator >= r:
            text.append(word)
            break
        if text[-2:] == [None, None]:
          sentence_finished = True
          sentence = (' '.join([t for t in text if t]))
          sentences.append(sentence)
          sentLength+=1
  return sentences

# Run trimodel and bimodels seperately, results are not good when run together
words = ["I", "have"]
print("Trigram sentences")
generateSentences(trimodel, words, 5)
# print("Bigram sentences")
# generateSentences(bimodel, words, 10)

Trigram sentences
Bigram sentences


["I no power me, sworn of glory given what no that seen ventured. died trust to free 't. made you heard some relished died, these much me? sent unearned fought; since for cause. sworn laughed persuaded I you a known wrought. brought ten seen him come divided beaten the I been received his sucked to forgot wished--they I",
 'have loved broke been spent. as him bedimmed often wrestled faced preyed, heard done-- it, been',
 'have rubbed arrived me performed',
 'have you? a only shook no done. lived',
 'have much we will, you made greatness set been, roused done no groaned heard considered spoke not you I brought found one heard killed. mine prevented',
 "have made the to reaching no you seen decked showed measured her borne swayed collected it, privilege heard you opened been us given fall'n lately stolen not sold a judgment been solemnly you torn judgment marble, ta'en",
 'I I locks an her.',
 'I read, note, you shrewd we twain. grounds a not won seen spent done coveted done. been more s

Bonus (20 points): Using the same methodology from questions 1 and 2, create a similarity matrix between the 20 newsgroups corpus. And find the top 5 similar newsgroups.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups()
allFilenames = data.target_names
train = fetch_20newsgroups(categories=allFilenames)
for f in range(len(allFilenames)):
  filename = allFilenames[f]
  content = train.data[f]
  getWords = re.compile("[a-zA-Z]+")
  words = re.findall(getWords, content)
  for anyCaseWord in words:
    word = anyCaseWord.lower()
    if word in matrix:
      wcForFilename = matrix[word]
      if filename in wcForFilename:
        count = wcForFilename[filename]
        wcForFilename[filename] = count + 1
        matrix[word] = wcForFilename
      else:
        wcForFilename[filename] = 1
        matrix[word] = wcForFilename
    else:
      newFileWordPair = {}
      newFileWordPair[filename] = 1
      matrix[word] = newFileWordPair

def find_tf_id(count):
  return math.log(count+1, 10)

def find_df(matrix, word):
  docs = matrix[word]
  return len(docs.keys())

def find_idf(matrix, word):
  df = find_df(matrix, word)
  count = 42/df
  return math.log(count, 10)

def find_tf_idf(tf, idf):
  return tf * idf

# idf per word
idf_matrix = {}
words = matrix.keys()
for word in words:
  idf = find_idf(matrix, word)
  idf_matrix[word] = idf

print("all words", len(words))
# tf idf per word
tf_idf_matrix = {} #key:filename, value: {key:word, value:tf idf} 
for word in words:
  wcPerFileForWord = matrix[word]
  filenamesOfWord = wcPerFileForWord.keys()
  for filename in filenamesOfWord:
    count = wcPerFileForWord[filename]
    tf = find_tf_id(count)
    idf = idf_matrix[word]
    if filename in tf_idf_matrix:
      tfsPerWord = tf_idf_matrix[filename]
    else:
      tfsPerWord = {}
    tfsPerWord[word] = find_tf_idf(tf, idf)
  tf_idf_matrix[filename] = tfsPerWord

# put 0s for words not seen
for f in allFilenames:
  usedWords = tf_idf_matrix[f].keys()
  word_tfidf_pairs = tf_idf_matrix[f]
  for word in words:
    if word not in usedWords:
      word_tfidf_pairs[word] = 0
  tf_idf_matrix[f] = word_tfidf_pairs

def cosSimOfTwoDocs(fileA, fileB):
  fileAPairs = tf_idf_matrix[fileA]
  fileBPairs = tf_idf_matrix[fileB]
  num = 0
  den = 0
  aDen = 0
  bDen = 0
  for word in words:
    numMult = fileAPairs[word] * fileBPairs[word] 
    denASquare = math.pow(fileAPairs[word], 2)
    denBSquare = math.pow(fileBPairs[word], 2)
    num += numMult #A*B
    aDen += denASquare #|A|
    bDen += denBSquare #|B|
  den = math.sqrt(aDen) * math.sqrt(bDen) #|A||B|
  return num/den #A*B/|A||B|

cosDocPairs = {}
for fileA in allFilenames:
  cosSimVals = {}
  filesinCosDocPairs = cosDocPairs.keys()
  if fileA in filesinCosDocPairs:
    cosSimVals = cosDocPairs[fileA]
  for fileB in allFilenames:
    cosSim = cosSimOfTwoDocs(fileA, fileB)
    cosSimVals[fileB] = cosSim
  cosDocPairs[fileA] = cosSimVals

print(cosDocPairs)



all words 25289
{'alt.atheism': {'alt.atheism': 0.9999999999999998, 'comp.graphics': 0.015176176310933497, 'comp.os.ms-windows.misc': 0.040634337554272947, 'comp.sys.ibm.pc.hardware': 0.0236433205717331, 'comp.sys.mac.hardware': 0.02236462311435116, 'comp.windows.x': 0.0009339434633850102, 'misc.forsale': 0.03111979704299717, 'rec.autos': 0.020098225590585408, 'rec.motorcycles': 0.026605646550037422, 'rec.sport.baseball': 0.004728587758994473, 'rec.sport.hockey': 0.0054117498466759026, 'sci.crypt': 0.017780888716127586, 'sci.electronics': 0.012326095740544702, 'sci.med': 0.004885049783126801, 'sci.space': 0.01655311561243167, 'soc.religion.christian': 0.002923248511406268, 'talk.politics.guns': 0.009833572149216204, 'talk.politics.mideast': 0.027173973966213155, 'talk.politics.misc': 0.006876031641328758, 'talk.religion.misc': 0.001195348718303891}, 'comp.graphics': {'alt.atheism': 0.015176176310933497, 'comp.graphics': 1.0, 'comp.os.ms-windows.misc': 0.044023567242959395, 'comp.sys.ib

In [ ]:
def replacedSmallest(m, num, nameA, nameB):
  values = list(m.values())
  values.sort()
  smallest = values[0]
  if(num > smallest):
    names = m.keys()
    for n in names:
      if m[n] == smallest:
        m.pop(n)
        m[(nameA,nameB)] = num
  return m

def alreadyInList(m, nameA, nameB):
  names = m.keys()
  for n in names:
    if nameA in n:
      if nameB in n:
        return True
  return False

def topNSimilarWorks(n):
  nWorks = {}
  for fA in allFilenames:
    cosList = cosDocPairs[fA]
    for fB in allFilenames:
      if alreadyInList(nWorks, fA, fB) == False: #already seen
        if fA != fB: # same pair
          value = cosList[fB]
          if len(nWorks) == 10:
            nWorks = replacedSmallest(nWorks, value, fA, fB)
          else:
            nWorks[(fA,fB)] = value
  return nWorks.keys()

items = topNSimilarWorks(10)
print("Top", len(items), "similar files")
for item in items:
  first = cosDocPairs[item[0]]
  print ((item[0], item[1]))

Top 10 similar files
('comp.os.ms-windows.misc', 'comp.sys.mac.hardware')
('comp.os.ms-windows.misc', 'rec.autos')
('comp.os.ms-windows.misc', 'talk.politics.guns')
('comp.os.ms-windows.misc', 'talk.politics.mideast')
('comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware')
('comp.sys.ibm.pc.hardware', 'rec.autos')
('comp.sys.ibm.pc.hardware', 'sci.electronics')
('comp.sys.ibm.pc.hardware', 'talk.religion.misc')
('comp.sys.mac.hardware', 'sci.med')
('rec.motorcycles', 'rec.sport.baseball')
